In [1]:
# Read the real network connection data, convert it into a hyperedge connection and store the connection data
import networkx as nx
from itertools import combinations


def process_graphml(file_path):

    # 1-hyperedges
    G = nx.read_graphml(file_path)
    node_mapping = {node: int(node[1:]) + 1 for node in G.nodes()}
    G = nx.relabel_nodes(G, node_mapping)
    
    edge_count = len(G.edges())
    print(f"Total number of edges (number of paired connections): {edge_count}")
    
    with open('Bnode_pairs.txt', 'w') as f_pairs:
        written_pairs = set()
        for edge in G.edges():
            sorted_pair = tuple(sorted(edge))
            if sorted_pair not in written_pairs:
                f_pairs.write(f"{sorted_pair[0]} {sorted_pair[1]}\n")
                written_pairs.add(sorted_pair)
    
    hyperedges = set()
    # 2-hyperedges
    for triplet in combinations(G.nodes(), 3):
        a, b, c = sorted(triplet) 
        if G.has_edge(a, b) and G.has_edge(a, c) and G.has_edge(b, c):
            hyperedges.add((a, b, c))
    
    with open('Bhyperedges.txt', 'w') as f_hyper:
        for hyperedge in sorted(hyperedges):
            f_hyper.write(f"{hyperedge[0]} {hyperedge[1]} {hyperedge[2]}\n")
    
    print(f"Number of 2-hyperedges: {len(hyperedges)}")

file_path = "rhesus_brain_2.graphml"
process_graphml(file_path)

Total number of edges (number of paired connections): 628
Number of 2-hyperedges: 477


In [3]:
from numba import jit, prange
import numpy as np
import random

# ---------------------------------------------------------
# Load hyperedges (input is 1-based, convert to 0-based)
# ---------------------------------------------------------
def load_hyperedges(file_path):
    triangles_list = []
    with open(file_path, 'r') as file:
        for line in file:
            t0, t1, t2 = map(int, line.strip().split())
            triangles_list.append((t0-1, t1-1, t2-1))
    return triangles_list


# ---------------------------------------------------------
# Compute second-order degree of each node
# ---------------------------------------------------------
def creating_list_k2_nodes(N, triangles_list):
    list_k2_nodes = np.zeros(N, dtype=np.int64)
    for a, b, c in triangles_list:
        list_k2_nodes[a] += 1
        list_k2_nodes[b] += 1
        list_k2_nodes[c] += 1
    return list_k2_nodes


# ---------------------------------------------------------
# Compute global intra-order overlap
# ---------------------------------------------------------
def global_intra_order_overlap_minimization(N, triangles_list, list_k2_nodes):

    local_intra_order_overlap = np.ones(N, dtype=np.float64)

    for i in range(N):
        k2 = int(list_k2_nodes[i])
        if k2 <= 1:
            continue

        max_cardinality = 2 * k2 + 1
        min_cardinality = np.ceil((3 + np.sqrt(1 + 8 * k2)) / 2)

        union_set = set()
        for triple in triangles_list:
            if i in triple:
                union_set.update(triple)

        union_cardinality = len(union_set)

        local_intra_order_overlap[i] = 1 - (union_cardinality - min_cardinality) / (max_cardinality - min_cardinality)

    return np.dot(local_intra_order_overlap, list_k2_nodes.astype(np.float64)) / np.sum(list_k2_nodes)


# ---------------------------------------------------------
# Weighted sampling of 3 nodes according to (k2 + 1)^alpha
# ---------------------------------------------------------
def weighted_sample_nodes(N, list_k2_nodes, alpha=2.0):
    weights = (list_k2_nodes + 1)**alpha
    probs = weights / weights.sum()
    return np.random.choice(N, size=3, replace=False, p=probs)


# ---------------------------------------------------------
# Generate random hyperedges with weighted sampling
# ---------------------------------------------------------
def generate_random_transversals_weighted(N, transversal_n, alpha=2.0):

    triangles_set = set()
    new_triangles_list = []
    list_k2_nodes = np.zeros(N, dtype=int)

    while len(new_triangles_list) < transversal_n:

        nodes = weighted_sample_nodes(N, list_k2_nodes, alpha)
        triangle = tuple(sorted(nodes))

        if triangle not in triangles_set:
            triangles_set.add(triangle)
            new_triangles_list.append(triangle)

            for n in nodes:
                list_k2_nodes[n] += 1

    return new_triangles_list, list_k2_nodes


# ---------------------------------------------------------
# Main procedure
# ---------------------------------------------------------
file_path = 'Bhyperedges.txt'
N = 91

triangles_list = load_hyperedges(file_path)
count = len(triangles_list)
print("Original number of hyperedges =", count)

alpha = 1.0
new_triangles_list, k2_new = generate_random_transversals_weighted(N, count, alpha=alpha)

overlap_value = global_intra_order_overlap_minimization(
    N, new_triangles_list, np.array(k2_new, dtype=np.float64)
)

# Save in 1-based format
saved_triangles = np.array(new_triangles_list) + 1
np.savetxt('new_triangles_list.txt', saved_triangles, fmt='%d')

# ---------------------------------------------------------
# Output
# ---------------------------------------------------------
print("\nFirst 10 new hyperedges (0-based):")
for tri in new_triangles_list[:10]:
    print(tri)

print("\nSaved hyperedges (1-based):")
for tri in saved_triangles[:10]:
    print(tri)

print("\nFinal overlap =", overlap_value)
print("Second-order degree (first 20 nodes):", k2_new[:20])
print("Max second-order degree =", k2_new.max())


Original number of hyperedges = 477

First 10 new hyperedges (0-based):
(np.int32(44), np.int32(54), np.int32(61))
(np.int32(18), np.int32(29), np.int32(66))
(np.int32(1), np.int32(9), np.int32(57))
(np.int32(15), np.int32(39), np.int32(49))
(np.int32(44), np.int32(58), np.int32(80))
(np.int32(8), np.int32(54), np.int32(86))
(np.int32(10), np.int32(53), np.int32(59))
(np.int32(14), np.int32(48), np.int32(83))
(np.int32(1), np.int32(12), np.int32(27))
(np.int32(29), np.int32(41), np.int32(58))

Saved hyperedges (1-based):
[45 55 62]
[19 30 67]
[ 2 10 58]
[16 40 50]
[45 59 81]
[ 9 55 87]
[11 54 60]
[15 49 84]
[ 2 13 28]
[30 42 59]

Final overlap = 0.36609125495952016
Second-order degree (first 20 nodes): [ 8 55  2 13 13  1  4  6 12  1 11 12 27  1 24  4 25  7 17  5]
Max second-order degree = 56


In [7]:
# Calculate the hyperedge overlap degree of each node
import numpy as np
from numba import jit

def load_hyperedges(file_path):
    triangles_list = []
    
    with open(file_path, 'r') as file:
        for line in file:
            t0, t1, t2 = map(int, line.strip().split())
            triangles_list.append((t0, t1, t2))
    
    return triangles_list

@jit(nopython=True)
def intra_order_overlap(triangles_list, N):
    list_k2_nodes = np.zeros(N, dtype=np.float64)
    
    for a, b, c in triangles_list:
        list_k2_nodes[a] += 1
        list_k2_nodes[b] += 1
        list_k2_nodes[c] += 1
    
    local_intra_order_overlap = np.zeros(N, dtype=np.float64)
    
    for i in range(N):
        k2 = int(list_k2_nodes[i]) 
        
        if k2 <= 1:
            local_intra_order_overlap[i] = 0  
            continue
        
        max_cardinality = 2 * k2 + 1
        min_cardinality = np.ceil((3 + np.sqrt(1 + 8 * k2)) / 2)
        
        union_set = set()
        for triple in triangles_list:
            if i in triple:
                union_set.update(triple)
        
        union_cardinality = len(union_set)
        local_intra_order_overlap[i] = 1 - (union_cardinality - min_cardinality) / (max_cardinality - min_cardinality)    
    

    global_intra_order_overlap = np.dot(local_intra_order_overlap, list_k2_nodes.astype(np.float64)) / np.sum(list_k2_nodes)
    
    return global_intra_order_overlap, local_intra_order_overlap

# ---------------------------------------------------------
# Main procedure
# ---------------------------------------------------------

file_path = 'new_triangles_list.txt'  
N = 91  
triangles_list = load_hyperedges(file_path)

global_overlap, local_overlap = intra_order_overlap(triangles_list, N)

with open("new_local_intra_order_overlap.txt", "w") as f:
    for i in range(N):
        f.write(f"{i}: {local_overlap[i]:.4f}\n")


In [6]:
# Average first-order second-degree Maximum first-order second-degree
import numpy as np

N = 91  
pairs_file = "Bnode_pairs.txt"
triangles_file = "Bhyperedges.txt"

edges = np.loadtxt(pairs_file, dtype=int)
total_degree_1 = len(edges) * 2  
average_degree_1 = total_degree_1 / N

triangles = np.loadtxt(triangles_file, dtype=int)
total_degree_2 = len(triangles) * 3 
average_degree_2 = total_degree_2 / N

degree_1 = np.zeros(N, dtype=int)  
for edge in edges:
    degree_1[edge[0] - 1] += 1 
    degree_1[edge[1] - 1] += 1

degree_2 = np.zeros(N, dtype=int) 
for triangle in triangles:
    degree_2[triangle[0] - 1] += 1  
    degree_2[triangle[1] - 1] += 1
    degree_2[triangle[2] - 1] += 1

max_degree_1 = np.max(degree_1)
max_degree_2 = np.max(degree_2)

print(f"Average 1st-order degree (k₁): {average_degree_1:.4f}")
print(f"Average 2nd-order degree (k₂): {average_degree_2:.4f}")
print(f"Max 1st-order degree: {max_degree_1}")
print(f"Max 2nd-order degree: {max_degree_2}")


Average 1st-order degree (k₁): 12.7912
Average 2nd-order degree (k₂): 15.7253
Max 1st-order degree: 87
Max 2nd-order degree: 183


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
plt.close('all') 
mpl.use('Qt5Agg') 
plt.ion() 

plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['mathtext.fontset'] = 'cm'     

def read_node_pairs(path):
    with open(path) as f:
        return [tuple(map(int, line.split())) for line in f]

def read_hyperedges(path):
    with open(path) as f:
        return [tuple(map(int, line.split())) for line in f if len(line.split()) == 3]

def read_local_overlap(path):
    out = {}
    with open(path) as f:
        for line in f:
            idx, val = line.strip().split(':')
            out[int(idx)] = float(val)
    return out

def second_order_degree(tri_file, N):
    tris = np.loadtxt(tri_file, dtype=int)
    deg2 = np.zeros(N, int)
    for tri in tris:
        for n in tri:
            deg2[n-1] += 1  
    return deg2

pairs_file   = "Bnode_pairs.txt"
tri_old_file = "Bhyperedges.txt"
tri_new_file = "new_triangles_list.txt"
ov_old_file  = "local_intra_order_overlap.txt"
ov_new_file  = "new_local_intra_order_overlap.txt"

pairs      = read_node_pairs(pairs_file)
tri_old    = read_hyperedges(tri_old_file)
tri_new    = read_hyperedges(tri_new_file)
ov_old     = read_local_overlap(ov_old_file)
ov_new     = read_local_overlap(ov_new_file)

N = 91
deg2_old = second_order_degree(tri_old_file, N)
deg2_new = second_order_degree(tri_new_file, N)

# --------------------------------------------------
# network topology
# --------------------------------------------------
G_old, G_new = nx.Graph(), nx.Graph()
G_old.add_edges_from(pairs)
G_new.add_edges_from(pairs)

pos_old = nx.spring_layout(G_old, k=0.5, seed=1)
pos_new = nx.spring_layout(G_new, k=0.5, seed=2)

cmap  = plt.cm.viridis
norm  = plt.Normalize(0, 1)
col_old = [cmap(norm(ov_old.get(n, 0))) for n in G_old.nodes()]
col_new = [cmap(norm(ov_new.get(n, 0))) for n in G_new.nodes()]


fig, ax = plt.subplots(3, 2,
                       figsize=(16, 10),  
                       gridspec_kw={
                           'height_ratios': [2.5, 1, 1],
                           'hspace': 0.85,     
                           'wspace': 0.15    
                       },
                       sharey='row')
fig.subplots_adjust(
    top=0.97,
    bottom=0.13,
    left=0.125,
    right=0.9,
    hspace=0.2,
    wspace=0.08
)

# ---------- Line 1: Network ----------
def draw_net(ax_here, G, pos, tris, node_colors):
    nx.draw_networkx_nodes(G, pos, node_color=node_colors,
                           node_size=12, linewidths=0, ax=ax_here)
    nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.35, ax=ax_here)
    for tri in tris:
        pts = [pos[n] for n in tri]
        ax_here.add_patch(plt.Polygon(pts, edgecolor='limegreen',
                                      facecolor='limegreen', alpha=0.04))
    ax_here.axis('off')

draw_net(ax[0, 0], G_old, pos_old, tri_old, col_old)
draw_net(ax[0, 1], G_new, pos_new, tri_new, col_new)


sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cb = fig.colorbar(sm, ax=[ax[0, 0], ax[0, 1]],
                  orientation='horizontal',
                  fraction=0.04, pad=0.08)

cb.ax.text(0.5, 1.4, r'$T^{(2)}_{i}$',  
           fontsize=14, ha='center', va='bottom',
           transform=cb.ax.transAxes)

cb.ax.tick_params(labelsize=12)
fig.subplots_adjust(wspace=0.08)

# ---------- Line 2: Overlap histogram ----------
bins_ov = np.linspace(0, 1, 61)
w_old = np.ones_like(list(ov_old.values())) / len(ov_old)
w_new = np.ones_like(list(ov_new.values())) / len(ov_new)
colors = ['#3951A2', '#A80326']   # 自定义填充色
for i, (data, w, face_col) in enumerate([
        (ov_old.values(), w_old, colors[0]),
        (ov_new.values(), w_new, colors[1])]):

    n, bins, patches = ax[1, i].hist(list(data),
                                     bins=bins_ov,
                                     weights=w,
                                     edgecolor='gray',
                                     facecolor=face_col)

    for patch in patches:
        patch.set_edgecolor((0.5, 0.5, 0.5, 0.4)) 
    
    ax[1, i].set_xlabel(r'$T^{(2)}_{i}$', fontsize=14)
    ax[1, i].set_ylim(0, 0.6)
    ax[1, i].tick_params(labelsize=12)
    ax[1, i].grid(False)

ax[1, 0].set_ylabel(r'$\mathit{P}$', fontsize=14)

# ---------- Row 3: Second degree histogram ----------
bins_deg = np.arange(0, max(deg2_old.max(), deg2_new.max()) + 3, 3)
w_d1 = np.ones_like(deg2_old) / len(deg2_old)
w_d2 = np.ones_like(deg2_new) / len(deg2_new)

face_colors = ['#3951A2', '#A80326'] 

for i, (d, w, face_col) in enumerate([
        (deg2_old, w_d1, face_colors[0]),
        (deg2_new, w_d2, face_colors[1])]):
    
    n, bins, patches = ax[2, i].hist(d,
                                     bins=bins_deg,
                                     weights=w,
                                     edgecolor='gray',
                                     facecolor=face_col,
                                     align='left')
    

    for patch in patches:
        patch.set_edgecolor((0.5, 0.5, 0.5, 0.4))

    ax[2, i].set_xlabel(r'$k_i^{(2)}$', fontsize=14)
    ax[2, i].set_ylim(0, 0.4)
    ax[2, i].set_xticks([0, 90, 180])   
    ax[2, i].set_yticks([0.0, 0.5])     
    ax[2, i].tick_params(labelsize=12)
    ax[2, i].grid(False)

ax[2, 0].set_ylabel(r'$\mathit{P}$', fontsize=14)


labels = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
y_positions = [1.01, 1.3, 1.3] 

for i in range(3):
    for j in range(2):
        ax[i, j].text(0.01, y_positions[i], labels[i*2 + j],
                      transform=ax[i, j].transAxes,
                      fontsize=14, va='top', ha='left')


# plt.tight_layout()
fig.canvas.manager.window.setGeometry(1308, 131, 536, 580) 
# plt.savefig("fig2.png", dpi=300, bbox_inches='tight')  
plt.show()
